In [3]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                            outputCol='features')
output = assembler.transform(data)
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [12]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                345|
|   mean| 497.74784990747645|
| stddev|  79.61809498709687|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [15]:
lr = LinearRegression(featuresCol = 'features', labelCol='Yearly Amount Spent')
model = lr.fit(train_data)
results = model.evaluate(test_data)
results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  13.38085443912155|
| 11.398831958847154|
|  7.886716346686171|
|  6.687796401701121|
| -20.99462223268904|
|-0.5640108352262132|
|   22.6800140781952|
|  4.441540065226604|
|  2.360838636102244|
| -5.412936144864375|
|   4.47888263094984|
| -17.51640222434594|
| 17.016594018692388|
| -5.229828728758662|
| -6.286104268971201|
|-1.8257551123771805|
|-1.8007152590127475|
|  12.73796516279532|
|-5.0158859278650425|
|  9.359679002102325|
+-------------------+
only showing top 20 rows



In [16]:
results.rootMeanSquaredError

10.314667039836914

In [17]:
results.r2

0.982745977963715

In [18]:
## Making actual predictions
unlabeled_data = test_data.select('features')
predictions = model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|395.25949663350593|
|[30.7377203726281...| 450.3819102373827|
|[30.9716756438877...|486.75189341020655|
|[31.0472221394875...| 385.8096027873203|
|[31.1239743499119...| 507.9416760724548|
|[31.2606468698795...| 421.8906420921776|
|[31.2834474760581...| 569.1010753474723|
|[31.3584771924370...| 490.7344103842488|
|[31.3895854806643...|407.70877242388065|
|[31.4474464941278...| 424.0156782400884|
|[31.5316044825729...| 432.0367230984127|
|[31.5702008293202...| 563.4618943657508|
|[31.6098395733896...| 427.5289556324158|
|[31.7207699002873...| 544.0047622067816|
|[31.7242025238451...| 509.6739915569317|
|[31.8186165667690...|448.24442848251283|
|[31.9120759292006...|389.33543156472047|
|[31.9262720263601...| 379.4669682815311|
|[31.9453957483445...|  662.035809865517|
|[31.9549038566348...|430.63820093782465|
+--------------------+------------